# CRF+LSTM

keras 2.2.4

tensorflow 1.13

pip install git+https://www.github.com/keras-team/keras-contrib.git

In [72]:
import re
import os
import pandas as pd

In [53]:
char_vocab_path = "CRF/data/char_vocabs.txt" # 字典文件
#train_data_path = 'data/train_data/train_data_000' # 训练数据
#train_data_path = './data/train_data' # 训练数据
#test_data_path = 'data/train_data/train_data_000' # 测试数据

special_words = ['<PAD>', '<UNK>'] # 特殊词表示

# "BIO"标记的标签
#label2idx = {"O": 0,
#             "B-PER": 1, "I-PER": 2,
#             "B-LOC": 3, "I-LOC": 4,
#             "B-ORG": 5, "I-ORG": 6
#            }
label2idx = {'O': 0,
             'B-DISEASE': 1, 'B-DISEASE_GROUP': 2,
             'B-DRUG_DOSAGE': 3, 'B-DRUG_EFFICACY': 4,
             'B-DRUG_INGREDIENT': 5, 'B-DRUG_TASTE': 6,
             'B-FOOD_GROUP':7, 'B-PERSON_GROUP':8,
             'B-SYMPTOM':9, 'B-SYNDROME':10,
             'I-DISEASE': 11, 'I-DISEASE_GROUP': 12,
             'I-DRUG_DOSAGE': 13, 'I-DRUG_EFFICACY': 14,
             'I-DRUG_INGREDIENT': 15, 'I-DRUG_TASTE': 16,
             'I-FOOD_GROUP':17, 'I-PERSON_GROUP':18,
             'I-SYMPTOM':19, 'I-SYNDROME':20
            }

# 索引和BIO标签对应
idx2label = {idx: label for label, idx in label2idx.items()}

# 读取字符词典文件
with open(char_vocab_path, "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
char_vocabs = special_words + char_vocabs

# 字符和索引编号对应
idx2vocab = {idx: char for idx, char in enumerate(char_vocabs)}
vocab2idx = {char: idx for idx, char in idx2vocab.items()}

In [54]:
# 读取训练语料
def read_corpus(corpus_path, vocab2idx, label2idx):
    with open(corpus_path, encoding='utf-8') as fr:
        lines = fr.readlines()

    sent_, tag_ = [], []
    for letter in lines:
        [char,label,_] = re.split('\t|\n',letter)
        sent_.append(char)
        tag_.append(label)

    sent_ids = [vocab2idx[char] if char in vocab2idx else vocab2idx['<UNK>'] for char in sent_]
    tag_ids = [label2idx[label] if label in label2idx else 0 for label in tag_]
    return sent_ids, tag_ids

# 加载训练集
#train_datas, train_labels = read_corpus(train_data_path, vocab2idx, label2idx)
# 加载测试集
#test_datas, test_labels = read_corpus(test_data_path, vocab2idx, label2idx)


In [55]:
train_datas = []
train_labels = []
files = os.listdir('data/train_data')
for file in files:
    train_data_path_i = 'data/train_data/'+file
    train_datas_i, train_labels_i = read_corpus(train_data_path_i, vocab2idx, label2idx)
    train_datas.append(train_datas_i)
    train_labels.append(train_labels_i)
    #if i%10==0:
    #    print(i)

In [56]:
valid_datas = []
valid_labels = []
files = os.listdir('data/valid_data')
for file in files:
    valid_data_path_i = 'data/valid_data/'+file
    valid_datas_i, valid_labels_i = read_corpus(valid_data_path_i, vocab2idx, label2idx)
    valid_datas.append(valid_datas_i)
    valid_labels.append(valid_labels_i)

In [57]:
print(train_datas[50])
print([idx2vocab[idx] for idx in train_datas[50]])
print(train_labels[50])
print([idx2label[idx] for idx in train_labels[50]])

[1, 61, 77, 1, 1, 17, 181, 3093, 3817, 2654, 6214, 1959, 2177, 286, 6802, 5965, 519, 1408, 2644, 2102, 2732, 1842, 889, 2545, 3093, 3817]
['<UNK>', 'b', 'r', '<UNK>', '<UNK>', '3', '、', '治', '疗', '期', '间', '忌', '房', '事', '，', '配', '偶', '如', '有', '感', '染', '应', '同', '时', '治', '疗']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [58]:
import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Masking, Embedding, Bidirectional, LSTM, Dense, Input, TimeDistributed, Activation
from keras.preprocessing import sequence
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras import backend as K
K.clear_session()

EPOCHS = 50
BATCH_SIZE = 128
EMBED_DIM = 48
HIDDEN_SIZE = 16
MAX_LEN = 50
VOCAB_SIZE = len(vocab2idx)
CLASS_NUMS = len(label2idx)
print(VOCAB_SIZE, CLASS_NUMS)

print('padding sequences')
train_datas = sequence.pad_sequences(train_datas, maxlen=MAX_LEN)
train_labels = sequence.pad_sequences(train_labels, maxlen=MAX_LEN)
valid_datas = sequence.pad_sequences(valid_datas, maxlen=MAX_LEN)
valid_labels = sequence.pad_sequences(valid_labels, maxlen=MAX_LEN)
print('x_train shape:', train_datas.shape)
print('x_test shape:', valid_datas.shape)

train_labels = keras.utils.to_categorical(train_labels, CLASS_NUMS)
valid_labels = keras.utils.to_categorical(valid_labels, CLASS_NUMS)
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', valid_labels.shape)

## BiLSTM+CRF模型构建
inputs = Input(shape=(MAX_LEN,), dtype='int32')
x = Masking(mask_value=0)(inputs)
x = Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(x)
x = Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True))(x)
x = TimeDistributed(Dense(CLASS_NUMS))(x)#TimeDistributed层的作用就是把Dense层应用到这10个具体的向量上，对每一个向量进行了一个Dense操作
outputs = CRF(CLASS_NUMS)(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

model.compile(loss=crf_loss, optimizer='adam', metrics=[crf_viterbi_accuracy])
model.fit(train_datas, train_labels, epochs=EPOCHS, verbose=1, validation_split=0.1)

score = model.evaluate(valid_datas, valid_labels, batch_size=BATCH_SIZE)
print(model.metrics_names)
print(score)

6874 21
padding sequences
x_train shape: (6899, 50)
x_test shape: (1987, 50)
trainlabels shape: (6899, 50, 21)
testlabels shape: (1987, 50, 21)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
masking_1 (Masking)          (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 48)            329952    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 32)            8320      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 21)            693       
_________________________________________________________________
crf_1 (CRF)                  (None, 50, 21)            945      

6209/6209 [==============================] - 24s 4ms/step - loss: 2.9196 - crf_viterbi_accuracy: 0.9436 - val_loss: 3.0596 - val_crf_viterbi_accuracy: 0.8938
Epoch 42/50
6209/6209 [==============================] - 19s 3ms/step - loss: 2.9177 - crf_viterbi_accuracy: 0.9454 - val_loss: 3.0559 - val_crf_viterbi_accuracy: 0.8945
Epoch 43/50
6209/6209 [==============================] - 17s 3ms/step - loss: 2.9172 - crf_viterbi_accuracy: 0.9450 - val_loss: 3.0553 - val_crf_viterbi_accuracy: 0.8987
Epoch 44/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9157 - crf_viterbi_accuracy: 0.9464 - val_loss: 3.0587 - val_crf_viterbi_accuracy: 0.8969
Epoch 45/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9149 - crf_viterbi_accuracy: 0.9485 - val_loss: 3.0553 - val_crf_viterbi_accuracy: 0.8962
Epoch 46/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9139 - crf_viterbi_accuracy: 0.9489 - val_loss: 3.0592 - val_crf_viterbi_accuracy:

In [59]:
# save model
model.save("model/ch_ner_model.h5")

In [111]:
def get_valid_nertag(input_data, result_tags):
    result_words = []
    start, end =0, 1 # 实体开始结束位置标识
    tag_label = "O" # 实体类型标识
    number = 0
    for i, tag in enumerate(result_tags):
        if tag.startswith("B"):
            number += 1
            if tag_label != "O": # 当前实体tag之前有其他实体     
                result_words.append(('T'+str(number), tag_label, start, end,input_data[start: end]))
                #result_words.append(('T'+str(number), tag_label+' '+str(start)+' '+str(end),input_data[start: end])) # 获取实体
            tag_label = tag.split("-")[1] # 获取当前实体类型
            start, end = i, i+1 # 开始和结束位置变更
        elif tag.startswith("I"):
            temp_label = tag.split("-")[1]
            if temp_label == tag_label: # 当前实体tag是之前实体的一部分
                end += 1 # 结束位置end扩展
        elif tag == "O":
            if tag_label != "O": # 当前位置非实体 但是之前有实体
                #result_words.append(('T'+str(number), tag_label+' '+str(start)+' '+str(end),input_data[start: end])) # 获取实体
                result_words.append(('T'+str(number), tag_label, start, end,input_data[start: end]))
                tag_label = "O"  # 实体类型置"O"
            start, end = i, i+1 # 开始和结束位置变更
    if tag_label != "O": # 最后结尾还有实体
        number += 1
        result_words.append(('T'+str(number), tag_label, start, end,input_data[start: end]))
        #result_words.append(('T'+str(number),tag_label+' '+str(start)+' '+str(end),input_data[start: end])) # 获取结尾的实体
    return result_words

In [136]:
maxlen = 50
result = {}
test_data_path = 'data/chusai_xuanshou/'
for i in range(500):
    test_file = test_data_path+str(i+1000)+'.txt'
    with open(test_file, "r", encoding="utf8") as test:
        sentence = test.read()
    sentences = sentence.split('。')
    y_ner = []

    for sent in sentences:
        sent = sent.replace(' ','_')
        sent_chars = list(sent+'。')
        sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]

        sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
        y_pred = model.predict(sent2id_new)
        y_label = np.argmax(y_pred, axis=2)
        y_label = y_label.reshape(1, -1)[0]
        y_ner_ = [idx2label[i] for i in y_label][-len(sent_chars):]
        y_ner.extend(y_ner_)
    result_words = get_valid_nertag(sentence, y_ner)
    ans = []
    for res in result_words:
        number = res[0]
        #tag_start_end = res[1]
        tag = res[1]
        start = res[2]
        end = res[3]
        word = res[4].replace(' ','')
        ans.append('{}\t{} {} {}\t{}'.format('T'+str(len(ans)), tag,start, end, word))
        #ans.append(, tag, start,end, "".join(word))
        print('{}\t{} {} {}\t{}'.format('T'+str(len(ans)), tag,start, end, word))
    result[i+1000] = ans

T1	DRUG_EFFICACY 138 142	清热解毒
T2	DRUG_EFFICACY 143 147	化湿除带
T3	DRUG_EFFICACY 148 152	祛瘀止痛
T4	DISEASE 160 165	慢性盆腔炎
T1	DISEASE 36 39	糖尿病
T2	SYMPTOM 87 91	月经量少
T3	SYMPTOM 95 101	血虚萎黄后错
T4	SYMPTOM 102 106	血虚萎黄
T5	SYMPTOM 107 111	风湿痹痛
T6	SYMPTOM 112 116	肢体麻木
T7	DISEASE 116 119	糖尿病
T1	PERSON_GROUP 36 42	感冒发热病人
T2	DISEASE 51 54	高血压
T3	DISEASE_GROUP 55 58	心脏病
T4	DISEASE_GROUP 59 61	肝病
T5	DISEASE_GROUP 66 68	肾病
T6	SYMPTOM 90 94	月经紊乱
T7	SYMPTOM 109 111	眩晕
T8	PERSON_GROUP 152 155	过敏者
T9	PERSON_GROUP 158 163	过敏体质者
T10	PERSON_GROUP 192 194	儿童
T11	DRUG_DOSAGE 242 244	颗粒
T12	DRUG_TASTE 245 248	气微香
T13	DRUG_TASTE 249 252	味微苦
T14	SYMPTOM 282 286	烘热汗出
T15	SYMPTOM 287 291	头晕耳鸣
T16	SYMPTOM 297 301	五心烦热
T17	SYMPTOM 302 306	腰背酸痛
T18	SYMPTOM 307 311	大便干燥
T19	SYMPTOM 312 315	心烦易
T20	SYNDROME 387 389	2g
T21	SYNDROME 389 391	*1
T1	DISEASE 55 59	子宫肌瘤
T2	SYNDROME 59 63	气滞血瘀
T3	SYMPTOM 66 70	经期延长
T4	SYMPTOM 71 75	经量过多
T5	SYMPTOM 76 82	经色紫黯有块
T6	SYMPTOM 83 90	小腹或乳房胀痛
T7	DRUG_DOSAGE 145 148	硬胶囊
T8	DRUG_TASTE 160 16

T1	DISEASE 25 27	皮疹
T2	DRUG_EFFICACY 76 80	祛瘀生新
T3	DRUG_EFFICACY 81 85	消肿止痛
T4	SYMPTOM 120 124	小腹疼痛
T1	DISEASE 34 38	贫血
T2	SYNDROME 108 112	气血不足
T1	FOOD_GROUP 52 54	生冷
T2	FOOD_GROUP 55 57	油腻
T3	DRUG_DOSAGE 85 88	大蜜丸
T4	SYMPTOM 106 110	tr><
T5	SYMPTOM 111 115	rcl
T6	SYMPTOM 116 120	ss='
T7	SYMPTOM 121 125	olor
T8	SYMPTOM 146 150	h><t
T9	SYMPTOM 151 155	>补肾，
T10	SYMPTOM 156 160	寒，止痛
T11	SYMPTOM 161 165	用于肾经
T12	PERSON_GROUP 165 167	虚寒
T13	DRUG_EFFICACY 167 170	引起的
T14	DRUG_EFFICACY 171 173	腿酸
T15	DRUG_EFFICACY 174 178	，阳痿遗
T16	DRUG_EFFICACY 179 181	，小
T17	SYMPTOM 191 195	补肾，
T18	SYMPTOM 196 200	寒，止痛
T19	SYMPTOM 201 205	用于肾经
T20	SYMPTOM 206 210	寒引起的
T21	SYMPTOM 210 218	腰腿酸痛，阳痿遗
T22	SYMPTOM 219 223	，小便频
T23	SYMPTOM 224 228	，小腹冷
T24	PERSON_GROUP 391 393	4、
T25	DRUG_DOSAGE 447 451	敏者禁用
T26	DRUG_DOSAGE 452 455	过敏体
T1	DISEASE 33 36	盆腔炎
T2	SYMPTOM 39 43	带下量多
T3	SYMPTOM 44 48	下腹疼痛
T4	DRUG_DOSAGE 55 59	薄膜衣片
T5	FOOD_GROUP 72 74	香，
T6	FOOD_GROUP 75 77	苦。
T7	FOOD_GROUP 78 80	1
T8	DRUG_DOSAGE 280 282

T1	SYMPTOM 41 45	宿有瘕块
T2	SYMPTOM 60 64	行经腹痛
T3	DRUG_EFFICACY 87 91	月。
T4	SYMPTOM 101 105	用于妇人
T5	SYMPTOM 109 113	，妊娠后
T6	SYMPTOM 114 118	下不止，
T7	SYMPTOM 120 124	不安，或
T8	SYMPTOM 125 129	瘀经闭，
T9	SYMPTOM 132 135	痛，产
T1	DRUG_DOSAGE 4 8	薄膜衣片
T2	DRUG_TASTE 21 24	气微香
T3	DRUG_TASTE 25 28	味微甜
T4	DRUG_TASTE 29 31	略苦
T5	DISEASE_GROUP 32 34	
T6	SYMPTOM 53 57	小腹胀痛
T1	DRUG_EFFICACY 1 5	祛瘀生新
T2	DRUG_DOSAGE 70 72	颗粒
T3	DRUG_TASTE 73 75	昧甜
T4	DRUG_TASTE 76 78	微苦
T5	FOOD_GROUP 92 94	生冷
T6	SYNDROME 99 103	气血两虚
T7	SYMPTOM 106 110	月经量少
T8	SYMPTOM 111 115	色淡质稀
T9	SYMPTOM 118 122	头晕心悸
T10	SYMPTOM 123 127	疲乏无力
T11	DISEASE 138 141	高血压
T12	DISEASE_GROUP 142 145	心脏病
T13	DISEASE_GROUP 146 148	肾病
T14	DISEASE 149 152	糖尿病
T15	SYMPTOM 189 192	经量少
T16	PERSON_GROUP 202 207	青春期少女
T17	PERSON_GROUP 208 213	更年期妇女
T18	SYMPTOM 230 232	腹痛
T19	SYMPTOM 234 238	阴道出血
T20	PERSON_GROUP 297 302	过敏体质者
T21	PERSON_GROUP 330 332	儿童
T22	SYNDROME 375 377	血瘀
T23	SYMPTOM 380 384	月经不调
T1	SYMPTOM 10 14	经前腹痛
T2	SYMPTOM 15 19	男子少精
T3	DRUG_EFFIC

T1	SYNDROME 3 7	气血两虚
T2	SYMPTOM 10 14	月经不调
T3	SYMPTOM 30 34	月经不调
T4	SYMPTOM 37 43	月经周期错后
T5	SYMPTOM 44 48	行经量少
T6	DRUG_EFFICACY 83 87	方药物（
T7	SYNDROME 90 94	,国家基
T8	SYMPTOM 108 114	气养血，活血
T9	SYMPTOM 115 119	经。用于
T10	SYMPTOM 120 124	血两虚兼
T11	SYMPTOM 125 129	血瘀所致
T12	SYMPTOM 129 134	的月经不调
T13	DRUG_EFFICACY 134 137	。症见
T14	DRUG_EFFICACY 138 141	经周期
T1	SYMPTOM 58 62	除烦安神
T2	SYMPTOM 63 65	用于
T3	SYMPTOM 66 68	年期
T4	SYMPTOM 69 71	现的
T5	DISEASE 128 134	一日2～3次
T6	SYMPTOM 138 142	善妇女更
T7	SYMPTOM 143 145	期综
T8	SYMPTOM 146 148	症症
T9	SYMPTOM 149 151	，如
T10	SYMPTOM 152 157	热汗出，眩
T1	DISEASE 68 74	更年期综合症
T2	SYMPTOM 78 82	潮热汗出
T3	SYMPTOM 83 85	眩晕
T4	SYMPTOM 86 88	耳鸣
T5	SYMPTOM 89 91	失眠
T6	PERSON_GROUP 103 109	徽)有限公司
T7	DISEASE 117 120	，少进
T8	DISEASE_GROUP 121 124	腻。2
T9	DISEASE_GROUP 125 127	感冒
T10	DISEASE 128 131	热病人
T11	DISEASE_GROUP 132 134	宜服
T12	DISEASE_GROUP 135 138	。3.
T13	SYMPTOM 155 159	病等慢性
T14	SYMPTOM 173 175	4.
T15	PERSON_GROUP 241 246	无缓解，应
T16	PERSON_GROUP 274 276	品性
T17	DRUG_DOSAGE 316 3

T1	SYNDROME 58 62	清热除
T1	SYNDROME 47 49	血瘀
T2	SYMPTOM 52 56	月经不调
T3	SYMPTOM 59 63	经水量少
T4	DRUG_EFFICACY 78 82	祛瘀生新
T5	SYMPTOM 85 89	月经量少
T6	SYMPTOM 93 97	经来腹痛
T7	DISEASE_GROUP 103 105	
T1	SYMPTOM 3 7	经期腹痛
T2	SYMPTOM 52 56	乙类）
T3	FOOD_GROUP 153 155	粒，
T4	SYMPTOM 193 195	人参
T1	DRUG_EFFICACY 4 6	调经
T2	DRUG_EFFICACY 7 9	止痛
T3	SYMPTOM 12 16	气血凝滞
T4	SYMPTOM 17 21	小腹胀疼
T1	PERSON_GROUP 69 71	孕妇
T2	DRUG_EFFICACY 83 87	活血化瘀
T3	SYMPTOM 97 101	宿有血块
T4	SYMPTOM 105 109	漏下不止
T5	SYMPTOM 110 114	胎动不安
T6	SYMPTOM 116 120	血瘀经闭
T7	SYMPTOM 121 125	行经腹痛
T8	SYMPTOM 128 131	恶露不
T9	SYMPTOM 148 152	月。
T1	DRUG_EFFICACY 31 35	滋补气血
T2	DRUG_EFFICACY 36 40	调经舒郁
T3	SYMPTOM 43 47	月经不调
T4	SYMPTOM 48 52	经行腹痛
T5	SYMPTOM 53 57	崩漏下血
T6	SYMPTOM 58 62	赤白带下
T7	SYMPTOM 63 67	贫血衰弱
T8	SYMPTOM 68 72	血晕血脱
T9	SYMPTOM 73 77	产后诸虚
T10	SYMPTOM 78 83	骨蒸潮热
T11	DRUG_INGREDIENT 174 176	次半
T12	DRUG_EFFICACY 182 189	日2次，温开水
T13	DRUG_INGREDIENT 190 192	服。
T14	DRUG_INGREDIENT 193 195	方
T15	DRUG_INGREDIENT 196 198	人参
T16	DRUG_INGREDIENT 199 201	

...

T1	DRUG_EFFICACY 55 57	祛瘀
T2	SYMPTOM 65 69	恶露不行
T3	SYMPTOM 72 76	小腹冷痛
T4	SYMPTOM 82 86	形寒肢冷
T5	DRUG_EFFICACY 126 128	祛瘀
T6	DRUG_EFFICACY 129 131	止痛
T7	SYMPTOM 136 140	恶露不行
T8	SYMPTOM 143 147	小腹冷痛
T9	SYMPTOM 153 157	形寒肢冷
T10	PERSON_GROUP 157 159	儿童
T11	PERSON_GROUP 159 161	孕妇
T12	SYMPTOM 170 174	恶露不行
T13	SYMPTOM 192 196	阴道流血
T14	SYMPTOM 197 201	月经过多
T15	SYMPTOM 201 204	热
T16	DRUG_EFFICACY 465 469	纤维蛋白
T17	DRUG_EFFICACY 489 492	解聚，
T18	DRUG_EFFICACY 676 678	膜结
T19	PERSON_GROUP 745 747	化颗
T20	PERSON_GROUP 747 749	粒对
T21	PERSON_GROUP 773 775	萄球
T22	DRUG_EFFICACY 775 779	菌均有很
T23	SYMPTOM 780 784	的抑菌作
T1	DRUG_EFFICACY 1 5	祛瘀生新
T2	SYMPTOM 8 12	月经量少
T3	SYNDROME 35 37	血瘀
T4	SYMPTOM 40 44	月经不调
T5	SYMPTOM 47 51	经水量少
T6	PERSON_GROUP 78 80	孕妇
T7	PERSON_GROUP 105 107	孕妇
T8	DISEASE_GROUP 128 130	
T1	DRUG_EFFICACY 3 7	养血祛瘀
T2	SYNDROME 8 11	血热证
T3	DRUG_EFFICACY 19 23	收缩子宫
T4	DRUG_EFFICACY 81 85	、促进造
T5	DRUG_EFFICACY 135 139	，抑制醋
T6	SYNDROME 172 174	“血
T7	SYMPTOM 276 280	露不行或
T8	SYMPTOM 281 285	而不畅，
T1	

T1	DRUG_EFFICACY 41 45	活血调经
T2	SYNDROME 48 52	气血两虚
T3	SYMPTOM 66 72	月经周期错后
T4	SYMPTOM 73 77	行经量少
T5	SYMPTOM 78 82	精神不振
T6	SYMPTOM 83 87	肢体乏力
T7	DRUG_EFFICACY 95 98	补气血
T8	DRUG_EFFICACY 99 102	调月经
T9	SYNDROME 172 176	气血两虚
T10	SYMPTOM 179 183	月经不调
T11	SYMPTOM 186 192	月经周期错后
T12	SYMPTOM 193 197	行经量少
T13	SYMPTOM 198 202	精神不振
T14	SYMPTOM 203 207	肢体乏力
T1	DRUG_EFFICACY 14 17	补气血
T2	DRUG_EFFICACY 18 21	调月经
T3	DRUG_EFFICACY 52 56	益气养血
T4	DRUG_EFFICACY 57 61	活血调经
T5	SYNDROME 64 68	气血两虚
T6	SYMPTOM 82 88	月经周期错后
T7	SYMPTOM 89 93	行经量少
T8	SYMPTOM 94 98	精神不振
T9	SYMPTOM 99 103	肢体乏力
T10	SYNDROME 134 138	药物同时
T11	SYMPTOM 141 145	能会发生
T12	SYMPTOM 148 154	互作用，详情
T13	SYMPTOM 155 159	咨询医师
T14	SYMPTOM 160 164	药师。
T15	SYMPTOM 165 170	用于气血两
T1	SYMPTOM 33 37	月经量少
T2	SYMPTOM 41 46	经来腹痛
T3	SYNDROME 67 69	血瘀
T4	SYMPTOM 81 85	月经不调
T5	DISEASE_GROUP 115 117	基本
T1	DRUG_EFFICACY 3 7	养血祛瘀
T2	SYNDROME 15 18	血热证
T3	SYMPTOM 63 67	行而不畅
T4	SYMPTOM 68 72	夹有血块
T5	DRUG_EFFICACY 133 137	幅度和频
T6	DRUG_EFFICACY 148 152	产妇产后
T7	DRUG_

T1	SYNDROME 34 36	血瘀
T2	SYMPTOM 39 43	月经不调
T3	SYMPTOM 46 50	经水量少
T4	PERSON_GROUP 50 52	孕妇
T5	DISEASE_GROUP 80 82	
T6	DRUG_EFFICACY 94 98	祛瘀生新
T7	SYMPTOM 101 105	月经量少
T8	SYMPTOM 109 113	经来腹痛
T1	DRUG_EFFICACY 6 10	活血调经
T2	SYNDROME 13 17	气血两虚
T3	SYMPTOM 31 37	月经周期错后
T4	SYMPTOM 38 42	行经量少
T5	SYMPTOM 43 47	精神不振
T6	SYMPTOM 57 61	月经不调
T7	SYNDROME 61 67	血瘀气血两虚
T8	DISEASE_GROUP 112 114	师或
T9	SYNDROME 145 149	尚不明确
T10	SYMPTOM 152 156	吉林省正
T11	SYMPTOM 159 165	业有限公司
T12	SYMPTOM 166 170	口服。一
T13	SYMPTOM 171 175	6克，一
T14	SYMPTOM 176 180	2次。
T15	SYMPTOM 180 182	用
T16	DRUG_EFFICACY 182 185	于气血
T17	DRUG_EFFICACY 186 189	虚所致
T1	DRUG_EFFICACY 43 45	止痛
T2	SYMPTOM 48 52	月经不调
T3	SYMPTOM 53 57	经期腹痛
T4	PERSON_GROUP 57 59	孕妇
T5	SYMPTOM 76 80	月经不调
T6	SYMPTOM 81 85	经期腹痛
T7	PERSON_GROUP 85 87	
T8	PERSON_GROUP 87 89	孕妇
T1	PERSON_GROUP 59 61	孕妇
T2	PERSON_GROUP 61 63	孕妇
T3	PERSON_GROUP 76 78	孕妇
T4	SYMPTOM 97 101	月经量少
T5	DISEASE_GROUP 116 118	
T6	SYNDROME 138 140	血瘀
T7	SYMPTOM 143 147	月经不调
T8	SYMPTOM 150 154	经水量少
T1	

T1	PERSON_GROUP 29 31	孕妇
T2	DISEASE_GROUP 34 36	
T3	SYMPTOM 81 85	月经量少
T4	SYMPTOM 89 93	经来腹痛
T5	SYNDROME 128 130	血瘀
T6	SYMPTOM 140 144	经水量少
T1	DRUG_EFFICACY 17 20	补气血
T2	DRUG_EFFICACY 21 24	调月经
T3	SYMPTOM 27 31	头晕心慌
T4	SYMPTOM 32 36	疲乏无力
T5	SYMPTOM 37 41	月经量少
T6	SYMPTOM 45 51	经期后错头晕
T7	DRUG_EFFICACY 54 57	补气血
T8	DRUG_EFFICACY 58 61	调月经
T9	SYMPTOM 64 68	月经不调
T1	DRUG_EFFICACY 3 7	养血祛瘀
T2	SYNDROME 15 17	血热
T3	DRUG_EFFICACY 40 44	收缩子宫
T4	DRUG_EFFICACY 103 107	促进造血
T5	DRUG_EFFICACY 157 161	抑制醋酸
T6	SYNDROME 194 196	血瘀
T7	SYMPTOM 324 328	不行或行
T8	SYMPTOM 329 333	不畅，夹
T1	FOOD_GROUP 26 28	辛辣
T2	FOOD_GROUP 29 31	生冷
T3	PERSON_GROUP 36 43	感冒发热病人
T4	DISEASE 52 55	高血压
T5	DISEASE_GROUP 56 59	心脏病
T6	DISEASE_GROUP 60 62	肝病
T7	DISEASE_GROUP 67 69	肾病
T8	DISEASE_GROUP 70 73	慢性病
T9	PERSON_GROUP 94 99	青春期少女
T10	PERSON_GROUP 100 105	更年期妇女
T11	SYMPTOM 129 133	月经过少
T12	SYMPTOM 135 139	经期错后
T13	SYMPTOM 141 148	阴道不规则出血
T14	SYMPTOM 158 162	月经量多
T15	PERSON_GROUP 214 219	过敏体质者
T16	PERSON_GROUP 249 251	儿童
T17	DRUG_DO

T1	SYNDROME 17 21	气血两虚
T2	SYMPTOM 35 41	月经周期错后
T3	SYMPTOM 42 46	行经量少
T4	SYMPTOM 47 51	精神不振
T5	SYMPTOM 52 56	肢体乏力
T6	SYMPTOM 56 61	行经量少
T7	SYMPTOM 120 124	用于
T8	SYMPTOM 127 133	虚所致的月经
T9	DRUG_EFFICACY 163 166	明确。
T10	DRUG_EFFICACY 167 170	每丸
T1	DRUG_EFFICACY 8 12	活血化瘀
T2	SYMPTOM 22 26	宿有血块
T3	SYMPTOM 30 34	漏下不止
T4	SYMPTOM 35 39	胎动不安
T5	SYMPTOM 41 45	血瘀经闭
T6	SYMPTOM 46 50	行经腹痛
T7	SYMPTOM 113 117	。一次6
T8	SYMPTOM 118 122	，一日1
T1	DRUG_EFFICACY 22 26	理气止痛
T2	SYMPTOM 46 48	痛经
T3	SYMPTOM 49 53	小腹胀痛
T4	SYMPTOM 54 59	腰腿酸痛痛
T5	FOOD_GROUP 68 70	辛辣
T6	FOOD_GROUP 71 73	生冷
T7	PERSON_GROUP 79 85	感冒发热病人
T8	DISEASE 94 97	高血压
T9	DISEASE_GROUP 98 101	心脏病
T10	DISEASE_GROUP 102 104	肝病
T11	DISEASE_GROUP 109 111	肾病
T12	DISEASE_GROUP 112 115	慢性病
T13	PERSON_GROUP 136 141	青春期少女
T14	PERSON_GROUP 142 147	更年期妇女
T15	SYMPTOM 171 175	月经过少
T16	SYMPTOM 177 181	经期错后
T17	SYMPTOM 183 190	阴道不规则出血
T18	SYMPTOM 201 205	月经量多
T19	PERSON_GROUP 257 264	过敏体质过敏者
T20	PERSON_GROUP 267 272	过敏体质者
T21	PERSON_GROUP 302 304	儿童
T22	DRUG_DOS

In [137]:
#“实体类别”、“起始位置”、“结束位置”以空格分隔

In [138]:
for i in range(1000,1500):
    with open('data/submit/%d.ann'%i,'w', encoding='utf-8') as wr:
        wr.write('\n'.join(result[i]))

In [129]:
''.join(result[1001])

'T1\tDISEASE 36 39\t糖尿病\nT2\tSYMPTOM 87 91\t月经量少\nT3\tSYMPTOM 95 101\t血虚萎黄后错\nT4\tSYMPTOM 102 106\t血虚萎黄\nT5\tSYMPTOM 107 111\t风湿痹痛\nT6\tSYMPTOM 112 116\t肢体麻木\nT7\tDISEASE 116 119\t糖尿病\n'

In [116]:
#for i in range(1000,1500):
#    pd.DataFrame(result[i]).to_csv('data/submit/%d.ann'%i,
#                                      sep='\t',
#                                      header = None,
#                                      index = 0,
#                                      encoding = 'utf-8')